# Hypothesis to test:
### Removing objective sentences from reviews helps predict star rating from reviews

In [1]:
import numpy as np
import pandas as pd
import pickle
import math
from nltk.tokenize import sent_tokenize


from IPython.display import Markdown, display

In [2]:
import sys
sys.path.append('..')

In [61]:
# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_colwidth', -1)

# %autosave 50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# import gzip
# import math
# import random
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.ensemble import GradientBoostingRegressor, \
# GradientBoostingClassifier, RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import SGDClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix, \
# classification_report, make_scorer
# import statsmodels.api as sm

In [5]:
# From this project
# from utils import rmse, rmse_train_cv, classifier_report, confusion_rmse
from NLP import WordBag, AboutMovie

In [68]:
%reload_ext autoreload
from subjective_filter import SubjectiveFilter

## Configuration

In [7]:
# Subsampling from Amazon reviews
NB_SAMPLES = 360000 #4000  # up to 200k, then change the input file

data_path = '../../../datasets/'


In [8]:
# file_name = '360000_balanced_train_test_reviews.pkl'
file_name = '_balanced_pos_neg_train_test_reviews.pkl'

pickle_in = open(data_path + str(NB_SAMPLES) + file_name,"rb")
train_test_dic0 = pickle.load(pickle_in)

In [52]:
SAMPLE_FRACTION = 0.0004

test_dic = {'train': {}, 'test':{}}

for i in ['train','test']:
    for j in ['positive','negative']:
         test_dic[i][j] = train_test_dic0[i][j] \
            .iloc[:math.floor(len(train_test_dic0[i][j].index) * SAMPLE_FRACTION), :] \
            .drop(['reviewerName', 'helpful',
                   'summary', 'unixReviewTime', 'reviewTime'], axis=1)


In [53]:
test_dic['train']['positive']

,reviewerID,asin,reviewText,overall
1515619,A32244V7CQUBD6,B00005QFEK,This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love. It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.,4.0
1515627,A32244V7CQUBD6,B00005QFEN,"This episode pretty much has Hafron and Emmanuelle teleporting to different parts of the world and &quot;doing it&quot;. There is the continuing plot from an earlier episode of some group on Earth trying to track them down. That's the main reason for Emmanuelle and Hafron to jump to different parts of the world. Otherwise, this episode is mostly sex scenes.",4.0
1515636,A33KKMGGVLZ29T,B00005QFER,"This is an intimate concert of Robert Mirabal. Although I thought that it was, as I said, masterful, the sound, at times sounded a little muffled.The storytelling of the songs gave an insight of native culture and of Mirabal's own family stories and history.The Dance and Ee You Oo are my picks for the best songs, but they are all a joy to watch. The Rare Tribal Mob and the Mirabal Singers/Dancers are great and provide a mesmerising stage performance.Very enjoyable",4.0
1515678,A33P47VEH0YULL,B00005QG2N,"A well put together DVD for the ""Stinkiest band"" in the world. Easy to navigate, and with some pretty interesting side notes...a great collectors item for any fan of Cradle of Filth or Swedish metal.",4.0
1515690,ADG33WELAQRZJ,B00005QG2N,"Great show from the inventors of the extreme gothic metal genre !! 75'of pure mayhem with good visuals and a great sound, although the vocals are a little high in the mix for my taste and the guitar on the right channel is 10 times louder than the one on the left so I ended up switching it to mono to even things up. It's hard to give this more stars esp. when you've watched the PTSFirepower DVD first like I did, that one is so much better. The 5 bonus videoclips totally rule and round up the package making the purchase a worthy one.Now the mockumentary may not be suited to everyone's tastes, personally it bored me and know of a lot of people who feels it ruins the band's image. Well don't watch it then.",4.0
1515691,A3YXITJWFW4BW,B00005QG2N,"I rented this dvd, i didnt buy it. I also rented PanDamonAeon and Mannequin. This dvd has excellent very long concert footage. Its practically the same songs as on Livebait for the Dead. And then we have the rest of the dvd. Im sorry, i love CoF but that ""schockumentary"" thing was just sad. I could barely stand to watch it. It's stupid and boring, maybe if theyd shown more of the band in a more coherent manner...and it just goes on and on forever. This is a 2 hour plus dvd adn half of it is crap. The ""blair twit Project"" is slightly amusing, but mostly boring. The music videos are good though. I would still recommend buying this dvd just for the music videos and the concert footage. PanDamonAeon gave me more joy though.",4.0
1515700,A214NHULS3H7OX,B00005QG2N,"Excellent Sound, Excellent Picture, Tons and Tons of songs. Love the Scorched Earth Erotic video. The only reason I didn't give this 5 stars is because I don't think they let their female singer sing enough, it adds a nice goth touch. I also did not give it five stars because it would have been nice to have the ""her ghost in the fog"" video on here, and finally I thin Dani could have done better on some parts of the songs. For example: On ""her ghost in the fog"" (live version), he chooses to growl in that typical death metal voice, instead of talking in the deep gothic voice like he does on the album. By using this growly voice, I think he is cheese'n off some of the songs best parts. Anyway, these are minor gripes and this is a must have DVD, so definitely BUY IT!...",4.0
1515701,A2IZOU2G1QX0JD,B00005QG2N,"As most of my friends hate, I'm massively into Cradle of Filth. In the style of Napoleon Dynamite describing a Liger, 

In [54]:
total = 0
for i in ['train','test']:
    for j in ['positive','negative']:
        print(test_dic[i][j].shape)
        total += test_dic[i][j].shape[0]
total

(57, 4)
(57, 4)
(14, 4)
(14, 4)


142

In [55]:
# pickle_in = open(data_path
#                 + 'movie_reviews_144000Pos_Neg_Samples.pkl'
#                 , "rb")
# test_dic = pickle.load(pickle_in)

## Remove objective sentences for case B

In [56]:
OBJ_THRESHOLD = 0.8

In [57]:
%reload_ext autoreload
obj_path = '../obj_subj_dev/'
fit_obj_tf = obj_path + 'fit_tfidf_vectorizer_for_obj_subj_sentences_classification.pkl'
fit_obj_model = obj_path + 'GBC_300_0.5_5_0.88cv.pkl'
subj_filter = SubjectiveFilter(fit_obj_tf, fit_obj_model)

In [73]:
# for tt in ['train','test']:
#     for pn in ['positive','negative']:
#         print(tt,pn,test_dic[tt][np].shape)
total = 0
for tt in test_dic.values():
    for df in tt.values():
        display(df.head(1))
        total += df.shape[0]
        display(df.shape)
print(total)

,reviewerID,asin,reviewText,overall
1515619,A32244V7CQUBD6,B00005QFEK,This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love. It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.,4.0


(57, 4)

,reviewerID,asin,reviewText,overall
861729,A3CFT5LHFCB46K,630362572X,"In the book The Official Godzilla Compendium, King Kong Vs Godzilla is described as ""The Jaws of the Japanese film Industry"" or ""One of the great monster battles of cinema history"". It is referred to as the film that ""...Lifted Godzilla from the swelling ranks of interchangeable atomic monsters of the fifties and placed him among the pantheon of cinema creatures"". Thus, I had high expectations when I watched this on the Sci-Fi channel.All I have to say is: Wow! The special effects are incredible! The acting is top notch. The screenplay is well written. This film is truly a classic of cinema that will leave you thinking long after you leave the theater.Wait. That's my review for 2001: A Space Odyssey.Here's what I really think of King Kong vs. Godzilla.A thousand words cannot describe my dislike of the film. I know it's a Godzilla film, but did it need to be this bad? It starts off good, with an excellent piece of music at the beginning credits. But soon goes downhill as we are introduced to U.N Reporter Eric Carter. All he was put in for was to make the film longer and annoy us. He ""communicates"" with people in Japan via a satellite that looks more like a first grade science project than visual effects. He constantly tells us that ""Godzilla is heading for Japan"" or ""King Kong and Godzilla will battle."" Duh! What do you think King Kong and Godzilla will do? Have tea and crumpets?Speaking of Kong, I have seen costumes that look better than this Kong suit. I would also rather listen to Celine Dion and Elton John do a music number together than listen to Kong roar. He doesn't even open his mouth! Godzilla appears just fine, except for the fact he's afraid of thermal energy (Didn't he tear down an electrical fence in 1954 with ease?). Kong however gets power from the thermal energy. Gee, I would assume that all his fur would make him the perfect conductor of elctricity. Also, doesn't anyone else notice that the islands of the first Godzilla movies (The '50s to the '70s) always look the same? The main battles themselves are also letdowns.It was a grand idea that Toho had: Pitting two titans of cinema against each other. Too bad the Japanese did not use stop motion animation. Or make a Kong/Godzilla rematch. Or make better costumes or etc, etc. For a true classic of monster cinema, King Kong, the original Godzilla or Godzilla 1985 are better choices than this pile of ... If this was the film that made Godzilla famous, he should have been assassinated.Indeed this film will leave you thinking after you leave the theater: Why did I waste my time on this?(Note: Perhaps my criticism was a bit harsh. After all, this is the American version of the movie and we always [mess] with perfection. Maybe the Japanese version is better. Get that one if you can).",1.0


(57, 4)

,reviewerID,asin,reviewText,overall
1515665,A2D832OA6Q5ZAS,B00005QFFP,"What a pleasure to see this peerless diva perform -- she is the purest example of her art that I have ever known. Like a great actress, she dissolves into her role, and yet her voice and style are easily recognizable for the effortless simplicity with which she nails every note and figure. I wish they could have taped in color in those days, and I would have liked to see her perform more examples of the coloratura repertoire and less of the popular romantic themes. Overall, I am delighted with this video.",4.0


(14, 4)

,reviewerID,asin,reviewText,overall
861752,A175U4QHQUXC69,6303625800,The DVD would not play anything but the selection page for color or black and white version. Only the introduction music plays,1.0


(14, 4)

142


In [71]:
REMOVE = 'obj'
THRESHOLD = 0.8
CHUNK_SZ = 1

sent_dfs = {'train':{},'test':{}}
nb_sentences_removed = 0

for ttname, tt in test_dic.items():
    for pn, df in tt.items():
        df_list = []
        start = 0
        while start < df.shape[0]:
            end = start + CHUNK_SZ
            df2 = subj_filter.to_one_sent_per_row(
                    df.iloc[start:end,:])
            df3, removed = subj_filter.transform(
                    df2,
                    'reviewText', 
                    remove_above_threshold = THRESHOLD,
                    debug_level=0,
                    remove=REMOVE)
            df_list.append(df3)
            nb_sentences_removed += removed
            start = end

        sent_dfs[ttname][pn] = df_list.pop()
        while len(df_list) > 0:
            sent_dfs[ttname][pn] = pd.merge(df_list.pop(), 
                                            sent_dfs[ttname][pn], how='outer')
#         res[ttname][pn] = reduce(lambda x, y: pd.merge(x, y, on = ['reviewerID','asin']), dfList)


Sentences removed:


../subjective_filter.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentence'] = df['reviewText'].map(sent_tokenize)


,reviewerID,asin,overall,reviewText
3,A33P47VEH0YULL,B00005QG2N,4.0,"A well put together DVD for the ""Stinkiest band"" in the world."
18,A20EEWWSFMZ1PN,B00005QIVV,4.0,"Based on Hugh Wiley's inscrutable Oriental detective Mr. Wong, Boris Karloff is called upon by his detective friend to solve the murder of undercover detective Dan O'Grady that includes smuggling."
21,A3NKLBL5XYHRH1,B00005QJHJ,4.0,"This concert shows Bonnie's versatility as a performer, from her brand of country-rock to blues to pure vocalist."
27,AGFO6RNCAAT09,B00005QJHL,4.0,"In intimate conversation with daughter Mary (whose husband directed), Paul McCartney discusses life after the Beatles, and how his next challenge was to form a band, pack up the family, and hit the road."
70,A28FO9TRAR77PM,B00005QIVG,4.0,Jordan Chan is a well respected hitman who wants to go into legit businesses with his crooked homies.
79,A3LU79BYMWZEW9,B00005QJHJ,4.0,I've caught her in all kinds of venues and she's never disappointed.
86,A1WLZYEOIL1HLT,B00005QJHL,4.0,"He is very playful and relaxed, given his familiarity with the interviewer."
95,A2EYZTJCR9YC9Y,B00005QJHV,4.0,She meets the other choir members and friendships and relationships develop.
96,A2CG0S6KDMWHMK,B00005QJHV,4.0,"He is a serious musician, but one who is respectful of the amateurs he is conducting."
121,A2IZOU2G1QX0JD,B00005QG2N,4.0,"Their music is among the worst in the industry, they suck in a way few bands can meaningfully aspire to."


#### => Removed 52 (13%) obj sentences

Sentences removed:


...

#### => Removed 8 (15%) obj sentences

../subjective_filter.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentence'] = df['reviewText'].map(sent_tokenize)


Sentences removed:


,reviewerID,asin,overall,reviewText
19,A3MV1KKHX51FYT,6303626475,1.0,"It was a dark and stormy night, a couple is driving on a muddy country road through the Welsh mountains."
33,A3MV1KKHX51FYT,6303626475,1.0,Their touring car does not have side curtains.
36,AKJOWY6V1EJPJ,6303631940,1.0,The Rani's plan is just complete gibberish (and if she's so super-intelligent why doesn't it dawn on her to build a rocket with an adjustable trajectory?
47,A3MV1KKHX51FYT,6303626475,1.0,"They are lost, and pull into a house with lights."
50,AKJOWY6V1EJPJ,6303631940,1.0,"), the Teraps are wasted, Mel is irritating in her every scene and the music is just dire."
103,A3MV1KKHX51FYT,6303626475,1.0,Then a knock on the door brings another couple to this house; they are also lost.
123,A30S01BP2NHY80,630363270X,1.0,"In other words, ignorance leads to ignorance and so on.This review, ((comment)), is no intended to heed my frustrations against the other reviewers, but at the fact there is no information to display the quality this DVD is in."
145,A3MV1KKHX51FYT,6303626475,1.0,]When they go to fetch a lamp they hear a strange sound.


#### => Removed 8 (9%) obj sentences

In [ ]:
display(Markdown('### {} sentences removed'.format(nb_sentences_removed)))

In [38]:
df1 = sent_dfs['train']['positive']
df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]

/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,reviewerID,asin,reviewText,overall


In [39]:
for ttname, tt in sent_dfs.items():
    for pn, df in tt.items():
        display(df.head(3))

,reviewerID,asin,reviewText,overall
0,A09409663AZ10ZKCJTTL7,959,My husband liked it very much.,4.0
1,A0999509ZRK5LA9PJ22N,811,well written story and well acted by players that you know and enjoy. i think this movie should have gotten better reviews when it first came out.,4.0
2,A100JCBNALJFAW,4570,"wow this movie blew me away, i saw it with a mind set that i was gonna hate but i didnt it was really good, and it had some comedy in it as well.now there are a few part that are a little cheap but its a fine damn movie. besides what other movie has a giant bunny?",4.0


,reviewerID,asin,reviewText,overall
0,A100JCBNALJFAW,3150,this is crap even if it was tittled something different it still would suck. there is no myers in this film. halloween 4 is the sequel to halloween 2 not 3,1.0
1,A100KCL8MFZPB0,1601,"The movie listed isn't the one you sent me. NOT SATISFIED and not going to waste my time on you again!!!!! I'll not ever order from Amazon again, and please don't send me these useless emails again!",1.0
2,A102M65QPGVHXI,4661,"This &quot;c&quot; movie had no story line, terrible acting bad correography, and nothing of value to offer viewers. As a Wing Chun practitioner and avid martial arts movie goer, I would like to offer some quality films with skillful actors and great fight scenes. This is the absolute worst (martial arts or otherwise) film I've ever had the displeasure of watching. Any viewers should be reimbursed for their time wasted. Have fun! I would recommend this film to no one.",1.0


,reviewerID,asin,reviewText,overall
0,A03950922R6OEJTOJ49IY,2042,Not really my show but I've watched a few and it is humorous. Was a great gift for my wife.,4.0
1,A0830356H8VW7AMXVVZB,3186,I bought it for my dad as a gift and he cantstop sharing with his friends and co-workers.Its ok for the value...(btwn $25-$30).,4.0
2,A100RW34WSLTUW,42,"I went away a little disappointed. This was predictable, and well done. It was good, however it could have been written better.",4.0


,reviewerID,asin,reviewText,overall
0,A01174011QPNX7GZF4B92,4252,"I was pleased with the condition it was in when I received it, but this movie is awful. Even White as the villain couldn't make up for the crap they shove down your throat in this movie.",1.0
1,A08280701W5RIXUPR6BZT,513,I never gotr it and Amazon still charged me for it and then declined my card and now what do i think i think this whole thing stinks and i want my money back,1.0
2,A100HB5G1JLUJ7,1915,This movie is so bad that I had to throw it in the trash to give it a decent burial! Do not buy this movie!!!,1.0


In [40]:
# df1 = sent_dfs['train']['positive']
# df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]
# df1

In [41]:
subj_dfs = sent_dfs

In [44]:
total = 0
old_total = 0
for tt in ['train', 'test']:
    for pn in ['positive', 'negative']:
        print('\n {} {}'.format(tt,pn))
        tot = test_dic[tt][pn].shape[0]
        new = subj_dfs[tt][pn].shape[0]
        dropped = tot - new
        print('Dropped: {0} ({1:.1%})'.format(dropped, dropped/tot))
        total += new
        old_total += tot
print('\nOld total:', old_total)
print('New total:', total)
print('Removed a total of:', old_total - total, 'reviews')


 train positive
Dropped: 1238 (2.1%)

 train negative
Dropped: 1152 (2.0%)

 test positive
Dropped: 302 (2.1%)

 test negative
Dropped: 272 (1.9%)

Old total: 144000
New total: 141036
Removed a total of: 2964 reviews


In [45]:
pickle_out = open(data_path
                    + 'reviews_wout_most_subj_' + str(int(round(DROP_RATIO * 100))) + '_B.pkl'
                    , "wb")
pickle.dump(subj_dfs, pickle_out)
pickle_out.close()

## Prep A_dic

In [46]:
A_dic = {'train': {}, 'test':{}}

for tt in ['train', 'test']:
    for pn in ['positive', 'negative']:
        df = test_dic[tt][pn]
        A_dic[tt][pn] = df[df['asin'].isin(subj_dfs[tt][pn]['asin'])]

In [47]:
total = 0
old_total = 0
for tt in ['train', 'test']:
    for pn in ['positive', 'negative']:
        print('\n {} {}'.format(tt,pn))
        tot = A_dic[tt][pn].shape[0]
        new = subj_dfs[tt][pn].shape[0]
        dropped = tot - new
        print('Dropped: {0} ({1:.1%})'.format(dropped, dropped/tot))
        total += new
        old_total += tot
print('\nOld total:', old_total)
print('New total:', total)
print('Removed a total of:', old_total - total, 'reviews')


 train positive
Dropped: 0 (0.0%)

 train negative
Dropped: 0 (0.0%)

 test positive
Dropped: 0 (0.0%)

 test negative
Dropped: 0 (0.0%)

Old total: 141036
New total: 141036
Removed a total of: 0 reviews


In [48]:
pickle_out = open(data_path
                    + 'reviews_wout_most_subj_' + str(int(round(DROP_RATIO * 100))) + '_A.pkl'
                    , "wb")
pickle.dump(A_dic, pickle_out)
pickle_out.close()